## Data Cleaning & Rules

### Zweck

Dieses Notebook bereinigt den exportierten Subgraphen (bis zu **300.000 Tracks**) und erzwingt einen **stabilen Data Contract**.
Ziel ist eine **konsistente und ML-fähige Clean-Layer**, die reproduzierbar für alle weiteren Analysen und Modelle verwendet wird.

---

### Vorgehen

* Vereinheitlichung von Datentypen und Strukturen
* Regelbasierte Bereinigung (Wertebereiche, Konsistenz, Duplikate)
* Integritäts- und Plausibilitätsprüfungen
* Deterministische, reproduzierbare Transformationen

---

### Input

* Exportierte CSVs aus dem Sampling/EDA (`interim/…`)

### Output

* Bereinigte CSVs (Inspektion)
* Parquet-Dateien (Training)
* JSON-Report mit angewandten Regeln und Statistiken

---

### Ergebnis

Eine **validierte, konsistente und reproduzierbare Datenbasis** als Grundlage für Feature Engineering und Modellierung.


## Imports

In [19]:
from __future__ import annotations
from typing import Dict
import numpy as np
import pandas as pd
from pathlib import Path
from utils.cleaning.core import (
    POLICY,
    snake_case,
    run_cleaning_pipeline,
    id_set, enforce_optional_fk_as_na, clean_bridge_fks,
    build_profiles, PROFILE_SPECS,
    apply_rule_stages, run_quality_gates, save_clean_layer_parquet, compute_rowcount_delta, write_cleaning_report
)
from utils.config.settings import RANDOM_SEED


np.random.seed(RANDOM_SEED)
pd.set_option("display.max_columns", 250)
pd.set_option("display.max_rows", 40)
pd.set_option("display.width", 160)

pd.options.mode.copy_on_write = True

import utils.core.paths as paths

importlib.reload(paths)

SAMPLE_NAME = paths.load_sample_name()
PATHS = paths.make_paths(SAMPLE_NAME)
paths.ensure_dirs(PATHS)


## Load CSV Exports

In diesem Schritt werden die zuvor exportierten **CSV-Dateien** in den
Speicher geladen.
Die CSVs repräsentieren den **verbundenen Subgraphen** des aktuellen Samples
und bilden die Rohbasis für alle nachfolgenden Cleaning- und Validierungsschritte.

Beim Laden wird **noch keine inhaltliche Bereinigung** durchgeführt –
Ziel ist ausschließlich das **strukturgetreue Einlesen** der Daten, sodass
alle Transformationen explizit und nachvollziehbar in den folgenden Schritten
erfolgen können.

In [20]:
EXPECTED_FILES = {
    "tracks": "tracks.csv",
    "audio_features": "audio_features.csv",
    "albums": "albums.csv",
    "artists": "artists.csv",
    "genres": "genres.csv",
    "r_albums_tracks": "r_albums_tracks.csv",
    "r_track_artist": "r_track_artist.csv",
    "r_artist_genre": "r_artist_genre.csv",
    "r_albums_artists": "r_albums_artists.csv",
}


def load_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, low_memory=False)
    df.columns = [snake_case(c) for c in df.columns]
    return df


raw: Dict[str, pd.DataFrame] = {}
missing = []

for table, fname in EXPECTED_FILES.items():
    fp = PATHS.raw_dir / fname
    if fp.exists():
        raw[table] = load_csv(fp)
    else:
        missing.append(table)

print(" Loaded tables:", list(raw.keys()))
if missing:
    print("Missing CSV exports:", missing)

{k: v.shape for k, v in raw.items()}

 Loaded tables: ['tracks', 'audio_features', 'albums', 'artists', 'genres', 'r_albums_tracks', 'r_track_artist', 'r_artist_genre', 'r_albums_artists']


{'tracks': (300000, 10),
 'audio_features': (299954, 15),
 'albums': (195938, 6),
 'artists': (187440, 4),
 'genres': (5455, 1),
 'r_albums_tracks': (340898, 2),
 'r_track_artist': (407296, 2),
 'r_artist_genre': (194023, 2),
 'r_albums_artists': (224955, 2)}

## Drop Useless / High-Missing Columns

Spalten ohne oder mit kaum nutzbarer Information werden frühzeitig entfernt,
um die **Datenstruktur zu vereinfachen** und **Rauschen** für nachgelagerte
Analysen und Modelle zu reduzieren.

Dabei gelten folgende Prinzipien:
- Spalten mit **100 % Missing Values** enthalten keine Information → werden entfernt.
- **High-Cardinality-Textfelder** (z. B. URLs) werden nicht direkt als Features genutzt.
  Stattdessen werden – falls sinnvoll – **binäre Indikatoren** abgeleitet
  (z. B. `has_preview`), um das enthaltene Signal zu erhalten.
- Technische oder redundante Felder ohne analytischen Mehrwert werden entfernt.

Dieses Vorgehen reduziert Speicherbedarf und Modellkomplexität, ohne relevante
Information zu verlieren.

In [21]:
if "tracks" in raw:
    raw["tracks"] = raw["tracks"].drop(columns=["is_playable"], errors="ignore")

if "albums" in raw:
    raw["albums"] = raw["albums"].drop(columns=["album_group"], errors="ignore")

if "tracks" in raw and "preview_url" in raw["tracks"].columns:
    raw["tracks"]["has_preview"] = raw["tracks"]["preview_url"].notna().astype("int8")
    raw["tracks"] = raw["tracks"].drop(columns=["preview_url"], errors="ignore")

## Applying the Cleaning

Das Cleaning wird als **deterministische Pipeline** auf alle vorhandenen Tabellen
angewendet.
Jede Tabelle wird nur dann verarbeitet, wenn sie im aktuellen Sample existiert.

Dabei werden:
- tabellenspezifische **Cleaner** ausgeführt,
- Regeln konsistent durchgesetzt,
- und die Struktur der Daten vereinheitlicht.

Das Ergebnis ist eine **vollständig bereinigte Clean-Layer**, die als Grundlage
für Validierung, Reporting und nachfolgende Modellierungs-Schritte dient.

In [22]:
cleaned = run_cleaning_pipeline(raw)
{k: v.shape for k, v in cleaned.items()}

{'tracks': (300000, 9),
 'audio_features': (299954, 15),
 'albums': (195938, 6),
 'artists': (187440, 4),
 'genres': (5455, 1),
 'r_albums_tracks': (340898, 2),
 'r_track_artist': (407296, 2),
 'r_artist_genre': (194023, 2),
 'r_albums_artists': (224955, 2)}

## Referential Integrity Enforcement

Nach dem tabellenspezifischen Cleaning wird die **referenzielle Integrität** zwischen
Entitäten und Bridge-Tabellen erzwungen, um eine **konsistente Graph-/Join-Struktur**
sicherzustellen.

### Bridge-Tabellen (Orphan-Removal)
Für alle Many-to-Many-Beziehungen werden **verwaiste Relationen (Orphans)** entfernt:
- Eine Zeile in einer Bridge-Tabelle bleibt nur erhalten, wenn **alle referenzierten IDs**
  in den jeweiligen Entity-Tabellen existieren (z. B. `track_id ∈ tracks`, `artist_id ∈ artists`).

Dadurch wird verhindert, dass spätere Joins:
- Datensätze künstlich aufblasen,
- fehlerhafte Verknüpfungen erzeugen,
- oder Modell-/EDA-Ergebnisse verfälschen.

### Optionale Foreign Keys (Soft Enforcement)
Einige Referenzen werden als **optional** behandelt (z. B. `tracks.audio_feature_id`):
- Ungültige Referenzen werden **nicht** durch Dropping der Track-Zeile behandelt,
  sondern auf `NaN` gesetzt.
- So bleibt der Track als Beobachtung erhalten, auch wenn die Zusatzinformationen fehlen.

**Ergebnis:**
Eine Clean-Layer mit **konsistenten Beziehungen**, ohne unnötigen Informationsverlust
durch aggressives Entfernen von Entitätszeilen.

In [23]:
# --- ID Sets (aus Clean-Layer) ---
track_ids = id_set(cleaned, "tracks", "track_id")
album_ids = id_set(cleaned, "albums", "id")
artist_ids = id_set(cleaned, "artists", "id")
genre_ids = id_set(cleaned, "genres", "id")
af_ids = id_set(cleaned, "audio_features", "id")

# --- Bridge-FK Specs ---
bridge_fk_specs = {
    "r_albums_tracks": (("album_id", album_ids), ("track_id", track_ids)),
    "r_track_artist": (("track_id", track_ids), ("artist_id", artist_ids)),
    "r_artist_genre": (("genre_id", genre_ids), ("artist_id", artist_ids)),
    "r_albums_artists": (("album_id", album_ids), ("artist_id", artist_ids)),
}

# --- Anwenden ---
if POLICY.drop_orphan_bridge_rows:
    cleaned = clean_bridge_fks(cleaned, bridge_fk_specs)

# Track -> audio_feature_id: invalid zu NA (nicht droppen)
cleaned = enforce_optional_fk_as_na(cleaned, "tracks", "audio_feature_id", af_ids)

{k: v.shape for k, v in cleaned.items()}

{'tracks': (300000, 9),
 'audio_features': (299954, 15),
 'albums': (195938, 6),
 'artists': (187440, 4),
 'genres': (5455, 1),
 'r_albums_tracks': (340898, 2),
 'r_track_artist': (407296, 2),
 'r_artist_genre': (194023, 2),
 'r_albums_artists': (218032, 2)}

## Post-Cleaning Vergleich (Before vs. After)

Die folgende Tabelle vergleicht zentrale Kennzahlen vor und nach dem Cleaning
und zeigt die Netto-Auswirkungen (ΔRows, ΔMemory, ΔDuplikate) pro Tabelle.

In [24]:
profiles_before = build_profiles(raw, PROFILE_SPECS)
profiles_after = build_profiles(cleaned, PROFILE_SPECS)

df_before = pd.DataFrame.from_dict(profiles_before, orient="index")
df_after = pd.DataFrame.from_dict(profiles_after, orient="index")

compare = df_after[["rows", "memory_mb", "duplicate_rows_on_keys"]].join(
    df_before[["rows", "memory_mb", "duplicate_rows_on_keys"]],
    lsuffix="_after", rsuffix="_before"
)

compare["rows_delta"] = compare["rows_after"] - compare["rows_before"]
compare["mem_delta_mb"] = compare["memory_mb_after"] - compare["memory_mb_before"]
compare["dup_delta"] = compare["duplicate_rows_on_keys_after"] - compare["duplicate_rows_on_keys_before"]

compare[["rows_before", "rows_after", "rows_delta", "mem_delta_mb", "dup_delta"]].sort_values("rows_delta")


,rows_before,rows_after,rows_delta,mem_delta_mb,dup_delta
r_albums_artists,224955,218032,-6923,-0.93,0


## Outlier & rule-based validation + flags

In diesem Schritt werden **Outlier nicht pauschal gelöscht** (z. B. via IQR), weil viele Spotify-Features **diskret**, **zero-inflated** oder **heavy-tail** verteilt sind. Ein IQR-Dropping wäre hier oft **zu aggressiv** und würde **gültige Spezialfälle** (z. B. Multi-Disc, Live-Tracks, sehr schnelle/langsame Songs) entfernen.

Stattdessen nutzen wir ein **rule-based (Data-Contract) Vorgehen**:

- **Domain-Verletzungen** (unmögliche Werte) → auf `NaN` setzen oder auf gültige Bereiche clippen
- **Extreme, aber plausible Werte** (Long-Tail) → **Quantile-Capping** (Upper Tail), kein Dropping
- **Signal erhalten** → zusätzliche **Flag-Features** (`is_*`) für Extremfälle

### Policy (konkret)
**Tracks**
- `popularity`: Clip auf **[0, 100]**
- `duration`: `<= 0` → `NaN`, **Upper-Tail Quantile-Cap** (z. B. 99.9%); Flag `is_long_track`
- `track_number`: `<= 0` oder `> 200` → `NaN`; Flag `is_tracknum_extreme`
- `disc_number`: `<= 0` → `NaN`; Flag `is_multidisc` (`> 1`); `> 10` → `NaN`; Flag `is_disc_extreme`

**Audio Features**
- `time_signature`: nur `{3,4,5}` gültig → sonst `NaN`; Flag `is_time_signature_rare`
- `tempo`: `<= 0` → `NaN`, **Upper-Tail Quantile-Cap** (z. B. 99.9%); Flag `is_tempo_extreme`
- `loudness`: außerhalb **[-60, 5]** → `NaN`; Flag `is_loudness_very_low` (`< -40`)
- `speechiness`: Clip auf **[0, 1]**; Flag `is_high_speech` (≥ 90%-Quantil)
- `instrumentalness`: Clip auf **[0, 1]**; Flag `is_instrumental` (≥ 0.5)
- `key`: nur **0–11** gültig → sonst `NaN`
- `mode`: nur **0–1** gültig → sonst `NaN`

**Artists**
- `followers`: `< 0` → `NaN`, **Upper-Tail Quantile-Cap** (z. B. 99.9%); Flag `is_followers_extreme`
- Zusatzfeature: `followers_log1p = log1p(followers)`
- `popularity`: Clip auf **[0, 100]**

**Albums**
- `release_date_parsed`: Jahr nur im Bereich **1900–2035** gültig → sonst `NaT`; Flag `is_release_year_invalid`
- `popularity`: Clip auf **[0, 100]**

So bleibt die Datenbasis **stabil und konsistent** für ML, ohne durch aggressives Dropping die Graph-/Join-Struktur oder seltene, aber valide Fälle zu verlieren.

In [25]:
cleaned = apply_rule_stages(cleaned)
print("Outlier rules applied: rule-based invalidation + quantile caps + flags (no IQR dropping).")


Outlier rules applied: rule-based invalidation + quantile caps + flags (no IQR dropping).


## Quality Gates (Data Contract Enforcement)

Nach Abschluss des Cleanings werden **Quality Gates** ausgeführt, um sicherzustellen,
dass der definierte **Data Contract** eingehalten wird.
Diese Gates dienen **nicht** der weiteren Bereinigung, sondern der **Validierung**
der finalen Clean-Layer.

### Geprüfte Invarianten

**Primärschlüssel (PK)**
- Alle PKs sind **nicht-null** und **eindeutig**
  - `tracks.track_id`
  - `audio_features.id`
  - `albums.id`
  - `artists.id`
  - `genres.id`

**Wertebereiche / Domänen**
- `popularity` ∈ **[0, 100]**
- `tracks.duration` > **0**
- Audio-Features im Bereich **[0, 1]** (`acousticness`, `energy`, …)

**Referenzielle Integrität (Bridges)**
- Alle Bridge-Tabellen enthalten **keine Orphan-Relations**
  - z. B. `album_id` ∈ `albums.id`, `track_id` ∈ `tracks.track_id`

### Verhalten bei Verstößen
Ein Verstoß gegen ein Quality Gate führt zu einem **harten Abbruch** des Notebooks.
Dadurch wird sichergestellt, dass **keine inkonsistenten Daten** in nachgelagerte
Schritte (Feature Engineering, Modelltraining) gelangen.

### Zweck
Quality Gates machen Annahmen über die Daten **explizit und überprüfbar** und
fungieren als **formale Schnittstelle** zwischen Cleaning und Modellierung.

In [26]:
id_sets = {
    "tracks": set(cleaned["tracks"]["track_id"].dropna().unique()) if "tracks" in cleaned else set(),
    "albums": set(cleaned["albums"]["id"].dropna().unique()) if "albums" in cleaned else set(),
    "artists": set(cleaned["artists"]["id"].dropna().unique()) if "artists" in cleaned else set(),
    "genres": set(cleaned["genres"]["id"].dropna().unique()) if "genres" in cleaned else set(),
}

run_quality_gates(cleaned, id_sets=id_sets)
print("All quality gates passed.")

All quality gates passed.


## Saving & Cleaning Report (JSON) — Audit & Reproducibility

Nach Abschluss des Cleanings wird ein **strukturierter Cleaning-Report** als JSON
gespeichert. Dieser Report dient der **Auditierbarkeit**, **Nachvollziehbarkeit**
und **Reproduzierbarkeit** der gesamten Cleaning-Pipeline.

### Enthaltene Informationen
- **Profiles (Before / After)**
  Tabellenstatistiken vor und nach dem Cleaning (Rows, Columns, Memory, Duplikate)

- **Rowcount Delta**
  Zeigt explizit, wie sich die Anzahl der Zeilen pro Tabelle durch das Cleaning
  verändert hat (z. B. durch Entfernen von Duplikaten oder Orphan-Relations)

- **Cleaning Notes / Policies**
  Dokumentiert zentrale Entscheidungen (z. B. FK-Policy, Popularity-Clipping,
  Duplikat-Strategie)

- **Run Metadata**
  Laufzeitinformationen (Timestamp, Environment, Policy, Pfade) zur vollständigen
  Reproduzierbarkeit

### Zweck
Der Cleaning-Report stellt sicher, dass jede erzeugte Clean-Layer **versionierbar,
prüfbar und reproduzierbar** ist und bildet die formale Schnittstelle zwischen
Data Cleaning und nachgelagerten Analyse- bzw. Modellierungs-Schritten.

In [27]:
# Clean Layer als Parquet speichern
save_clean_layer_parquet(cleaned, PATHS.clean_parquet_dir)
print("Clean layer saved to:", PATHS.clean_parquet_dir)


# Delta berechnen
rowcount_delta = compute_rowcount_delta(raw, cleaned)

# Notes (kurz halten)
notes = {
"bridge_policy": "drop orphan rows (referential integrity enforced)",
"track_audio_feature_fk_policy": "invalid audio_feature_id set to NA (tracks not dropped)",
"popularity_policy": "clipped to [0,100]",
"duplicate_policy": "kept the most complete row per PK",
"export_formats": "Parquet",
}


# Report schreiben
report_path = PATHS.reports_dir_cleaning / "cleaning_report.json"
write_cleaning_report(
report_path=report_path,
profiles_before=profiles_before,
profiles_after=profiles_after,
rowcount_delta=rowcount_delta,
notes=notes
)
print("Cleaning report written:", report_path)

Clean layer saved to: C:\GitHub\uni-project-metrics-and-data\data\processed\parquet\slice_001
Cleaning report written: C:\GitHub\uni-project-metrics-and-data\data\reports\cleaning\slice_001\cleaning_report.json
